<a href="https://www.kaggle.com/code/marinabalakina/dll30-dz4-4?scriptVersionId=155050799" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# ** Домашнее** задание по теме «Архитектуры свёрточных сетей»

Цель задания: изучить работу с готовыми моделями из torchvision.

Контекст

Вам необходимо подобрать базовую модель для работы по вашей задаче. Вы пробуете обучать различные модели на “ваших” данных. По результатам отберёте лучшую для дальнейшего обучения.

Задание

Вам необходимо провести эксперименты по начальному обучению различных моделей и сравнить результаты.

1.Возьмите датасет EMNIST из torchvision

2. Обучите на нём модели: ResNet 18, VGG 16, Inception v3, DenseNet 161 (с нуля по 10 эпох)

3. Сведите результаты обучения моделей (графики лоса) в таблицу и сравните их.


Задание со звездочкой*

* Выполните то же задание, используя датасет hymenoptera_data

Инструкция к выполнению задания

* Загрузите датасет, посмотрите примеры картинок в нём и проверьте наличествующие классы и их дисбаланс.

* Создайте модель текущего типа, используя интерфейс torchvision для нужного количества классов.

* Обучите модель с нуля 10 эпох. Фиксируйте значение функции потерь в список для последующего отображения.

Повторите пункты 2 и 3 для всех указанных вариантов моделей.

Формат сдачи работы

Прикрепите ссылку на готовое решение в личном кабинете. Работу можно отправлять в виде ссылки на python-ноутбук из GitHub, Google Colaboratory или аналогичных платформ. Не забудьте открыть доступ на просмотр и комментирование.

Критерии оценивания
По итогу выполнения задания вы получите зачёт.

Задание считается выполненным, если:

вы обучили каждую модель до некоторого улучшения качества

составлена таблица обучения для сравнения

Задание будет отправлено на доработку, если:

использованы не все типы моделей

не составлена сводная таблица с результатами

# Импорт библиотек и пользовательские функции

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [2]:
import torch
from torch import nn
import torchvision as tv # consists of popular datasets, model architectures, and common image transformations for computer vision - для работы с предобученными нейросетями
import time

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [5]:
def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    train_accuracy, train_losses, test_accuracy =[], [], []
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')
        train_accuracy.append(train_acc_sum / n)
        train_losses.append(train_l_sum / n)
        test_accuracy.append(test_acc)
    return train_accuracy, train_losses, test_accuracy

In [6]:
BATCH_SIZE = 256
# Переводим картинки в 224х224 и в тензор
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])
train_dataset = tv.datasets.EMNIST('.', split='mnist', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.EMNIST('.', split='mnist', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

# 2. Обучение моделей

In [7]:
net = tv.models.densenet161(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:03<00:00, 38.2MB/s] 


In [8]:
net

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [11]:
# Убираем требование градиента:
for param in net.parameters():
    param.requires_grad = False

In [12]:
net.classifier

Linear(in_features=2208, out_features=1000, bias=True)

In [13]:
net.classifier = nn.Linear(in_features=2208, out_features=10)

In [14]:
print("Params to learn:")
params_to_update = []
for name, param in net.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.weight
	 classifier.bias


In [15]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(params_to_update, lr=lr)

In [16]:
train_accuracy, train_losses, test_accuracy  = train(net, train_iter, test_iter, trainer, num_epochs)

Step 0. time since epoch: 2.463. Train acc: 0.137. Train Loss: 2.319
Step 10. time since epoch: 15.698. Train acc: 0.429. Train Loss: 1.937
Step 20. time since epoch: 28.838. Train acc: 0.574. Train Loss: 1.665
Step 30. time since epoch: 41.929. Train acc: 0.658. Train Loss: 1.455
Step 40. time since epoch: 55.066. Train acc: 0.706. Train Loss: 1.301
Step 50. time since epoch: 68.167. Train acc: 0.739. Train Loss: 1.181
Step 60. time since epoch: 81.308. Train acc: 0.762. Train Loss: 1.088
Step 70. time since epoch: 94.390. Train acc: 0.780. Train Loss: 1.013
Step 80. time since epoch: 107.514. Train acc: 0.794. Train Loss: 0.951
Step 90. time since epoch: 120.677. Train acc: 0.806. Train Loss: 0.896
Step 100. time since epoch: 133.765. Train acc: 0.816. Train Loss: 0.850
Step 110. time since epoch: 146.987. Train acc: 0.825. Train Loss: 0.811
Step 120. time since epoch: 160.138. Train acc: 0.832. Train Loss: 0.777
Step 130. time since epoch: 173.218. Train acc: 0.840. Train Loss: 0.74

In [17]:
df_results= pd.DataFrame(columns = ['model', 'train_accuracy', 'train_loss', 'test_accuracy','epoch'])
for i in range(10):
  df_results.loc[len(df_results.index)] = ['dencenet161',  train_accuracy[i], train_losses[i], test_accuracy[i], i]

In [18]:
df_results

,model,train_accuracy,train_loss,test_accuracy,epoch
0,dencenet161,0.879967,0.551938,0.9412,0
1,dencenet161,0.950650,0.207485,0.9563,1
2,dencenet161,0.961600,0.155470,0.9616,2
3,dencenet161,0.967067,0.130077,0.9641,3
4,dencenet161,0.970150,0.114286,0.9659,4
5,dencenet161,0.972467,0.103202,0.9672,5
6,dencenet161,0.974567,0.094830,0.9682,6
7,dencenet161,0.976017,0.088192,0.9685,7
8,dencenet161,0.977367,0.082741,0.9692,8
9,dencenet161,0.978733,0.078149,0.9694,9


In [19]:
df_results.to_csv('dencenet_161.csv', index=False)

In [ ]:
torch.cuda.empty_cache()
# model = tv.models.inception_v3(pretrained=True)

In [ ]:
model

In [ ]:
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

In [ ]:
lr, num_epochs = 0.001, 1
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train_accuracy, train_losses, test_accuracy  = train(model, train_iter, test_iter, trainer, num_epochs)

In [ ]:
for i in range(10):
  df_results.loc[len(df_results.index)] = ['inception_v3',  train_accuracy[i], train_losses[i], test_accuracy[i], i]

## 2.4. DenseNet 161

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = tv.models.densenet161(pretrained=True)

In [ ]:
model

In [ ]:
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.classifier

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)
train_accuracy, train_losses, test_accuracy  = train(model, train_iter, test_iter, trainer, num_epochs)

In [ ]:
for i in range(10):
  df_results.loc[len(df_results.index)] = ['densenet161',  train_accuracy[i], train_losses[i], test_accuracy[i], i]